# Sarcoma Stage to OMOP

Sarcoma Stage is mainly mapped to omop Measurement and perhaps to episode, as showed in https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=2128970248#gid=2128970248

In case it is to an episode, it should reference the overarching episode, right? Bur HN should be similar and it does not have any "episode".

# Doubts
As in episode event, we still have the doubt of where the disease extent and howit is represented

Regional nodal metastases
Soft tissue
distant lymph node
lung
bone
liver
pleura
peritoneum
brain
other viscera
unknown

These are BOOLEANS with VOCABULARY and variable as modifier, what should i do?

For Localised and Loco-regional do we have to change the values in the table episode, or what should be done here?

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
sarcomaStage_IDEA4RC = pd.read_csv("./IDEA4RC-data/sarcomaStageIDEA4RC.csv")
sarcomaStage_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Sarcoma Stage to Measurement/Event

Regional nodal metastases
Soft tissue
distant lymph node
lung
bone
liver
pleura
peritoneum
brain
other viscera
unknown

These are BOOLEANS with VOCABULARY and variable as modifier, what should i do?

For Localised and Loco-regional do we have to change the values in the table episode?

In [ ]:
df_tables=sarcomaStage_IDEA4RC

cols= {
    'Localised' : 0,
    'Loco-regional' : 0,
    'Regional nodal metastases': 36769269,
    'Soft tissue': 35225724,
    'distant lymph node': 36769243,
    'lung': 36770283,
    'bone': 36769301,
    'liver': 36770544,
    'pleura': 35226258,
    'peritoneum': 35226253,
    'brain': 36768862,
    'other viscera': 36769180,
    'unknown': 4129922
}
sql = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date,measurement_type_concept_id,value_as_concept_id,measurement_event_id)
    VALUES (%s, %s, %s,%s,%s,%s);
"""
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM episode_event e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1;

"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s;
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1;
"""
date=datetime.now()
measurement_type_concept_id=38000280
for idx, row in df_tables.iterrows():
    for column in cols.keys():
        if ((column=='Localised' and row[column]==32942) or (column=='Loco-regional' and row[column]==32943)):
            curIDEA.execute(query, (row['Episode Event Reference']))
            res=curIDEA.fetchone()
            person_id=res
            sql2="""
                UPDATE omopcdm.episode
                SET episode_concept_id = %s
                WHERE episode_id= %s;
            """
            curIDEA.execute(queryDate, (row['Episode Event Reference']))
            res=curIDEA.fetchone()
            dateAux=res
            cur.execute(sqlGetEpisode, (person_id,dateAux))
            episode_id_ref=cur.fetchone()
            cur.execute(sql2, (row[column],episode_id_ref))
            conn.commit()
        else:
            curIDEA.execute(query, (row['Episode Event Reference']))
            res=curIDEA.fetchone()
            person_id=res
            if row[column] in [4188539,4188540] :
                measurement_value = row[column]
            else:
                if row[column]==0:
                    measurement_value=4188540
                else:
                    measurement_value=4188539
            measurement_concept_id = cols.get(column)
            curIDEA.execute(queryDate, (row['Episode Event Reference']))
            res=curIDEA.fetchone()
            dateAux=res
            cur.execute(sqlGetEpisode, (person_id,dateAux))
            episode_id_ref=cur.fetchone()
            cur.execute(sql, (person_id, measurement_concept_id,date , measurement_type_concept_id,measurement_value,episode_id_ref))
            conn.commit()

